# Обратная задача динамики в пространстве

Для начала определим необходимый матан:

In [ ]:
from sympy import *
from IPython import display
init_printing()

class Vector(Expr):

    def __init__(self, x, y, z):
        super(Vector, self).__init__()
        self.x = x
        self.y = y
        self.z = z

    @staticmethod
    def zero():
        return Vector(0, 0, 0)

    @staticmethod
    def lerp(start, end, t):
        return start + t * (end + -1 * start)

    def normalized(self):
        return 1 / self.magnitude() * self

    def magnitude(self):
        return (self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5

    def times(self, other):
        return Vector(
            other * self.x,
            other * self.y,
            other * self.z
        )

    def cross(self, other):
        return Vector(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x,
        )

    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z

    def angle_to(self, other, axis):
        crossed = self.cross(other)
        collinearity = crossed.dot(axis)
        return arctan2(
            crossed.magnitude(),
            self.dot(other)
        ) * (1 if collinearity > 0.0 else -1)

    def __str__(self):
        return "[{}, {}, {}]".format(self.x, self.y, self.z)

    def __repr__(self):
        return "Vector[{}, {}, {}]".format(self.x, self.y, self.z)

    def __add__(self, other):
        if isinstance(other, Vector):
            return Vector(
                self.x + other.x,
                self.y + other.y,
                self.z + other.z
            )
        return Add(self, other)

    def __mul__(self, other):
        return self.times(other)

    def __rmul__(self, other):
        return self.times(other)
    
    def _repr_latex_(self):
        return r"$\left[{}, {}, {}\right]$".format(
            printing.latex(self.x),
            printing.latex(self.y),
            printing.latex(self.z),
        )
    
    def _latex(self, printer):
        return r"\left[{}, {}, {}\right]".format(
            printing.latex(self.x),
            printing.latex(self.y),
            printing.latex(self.z),
        )
    
    def simplify(self, **kwargs):
        return Vector(
            simplify(self.x, **kwargs),
            simplify(self.y, **kwargs),
            simplify(self.z, **kwargs)
        )
    
    def velocity(self, t):
        return sqrt(
            diff(self.x, t) ** 2 +
            diff(self.y, t) ** 2 +
            diff(self.z, t)
        )
    
    def __pow__(self, other):
        return Vector(
            self.x ** other,
            self.y ** other,
            self.z ** other
        )
    
    def diff(self, t):
        return Vector(
            diff(self.x, t),
            diff(self.y, t),
            diff(self.z, t)
        )


class Quaternion(Expr):

    def __init__(self, w, x, y, z):
        super(Quaternion, self).__init__()
        self.w = w
        self.x = x
        self.y = y
        self.z = z

    @staticmethod
    def identity():
        return Quaternion(1, 0, 0, 0)

    def rotate(self, vector):
        pure = Quaternion(0, vector.x, vector.y, vector.z)
        rotated = self * pure * self.conjugate()
        return Vector(rotated.x, rotated.y, rotated.z)

    def magnitude(self):
        return (self.w ** 2 + self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5

    def normalized(self):
        return 1 / self.magnitude() * self

    def dot(self, other):
        return self.w * other.w + self.x * other.x + self.y * other.y + self.z * other.z

    @staticmethod
    def slerp(start, end, t):
        dot = start.dot(end)
        if dot < 0.0:
            start = -1 * start
            dot = -dot
        if dot >= 1.0:
            return start
        theta = arccos(dot) * t
        q = (end + -dot * start).normalized()
        return cos(theta) * start + sin(theta) * q

    def conjugate(self):
        return Quaternion(self.w, -self.x, -self.y, -self.z)

    def multiply(self, other):
        return Quaternion(
            self.w * other.w - self.x * other.x - self.y * other.y - self.z * other.z,
            self.w * other.x + self.x * other.w + self.y * other.z - self.z * other.y,
            self.w * other.y - self.x * other.z + self.y * other.w + self.z * other.x,
            self.w * other.z + self.x * other.y - self.y * other.x + self.z * other.w
        )

    def plus(self, other):
        return Quaternion(
            self.w + other.w,
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    def times(self, other):
        return Quaternion(
            other * self.w,
            other * self.x,
            other * self.y,
            other * self.z
        )

    @staticmethod
    def from_angle_axis(angle, axis, unit=True):
        if not unit:
            axis = axis.normalized()
        c = cos(angle / 2)
        s = sin(angle / 2)
        return Quaternion(
            c,
            s * axis.x,
            s * axis.y,
            s * axis.z
        )

    def __str__(self):
        return "[{}, {}, {}, {}]".format(self.w, self.x, self.y, self.z)

    def __repr__(self):
        return "Quaternion[{}, {}, {}, {}]".format(self.w, self.x, self.y, self.z)

    def __mul__(self, other):
        if isinstance(other, Vector):
            return self.rotate(other)
        if isinstance(other, Quaternion):
            return self.multiply(other)
        if isinstance(other, MatrixSymbol):
            return Mul(self, other)
        return self.times(other)

    def __add__(self, other):
        return self.plus(other)

    def __rmul__(self, other):
        return self.times(other)
    
    def diff(self, t):
        return Quaternion(
            diff(self.w, t),
            diff(self.x, t),
            diff(self.y, t),
            diff(self.z, t)
        )
    
    def angular_velocity(self, t):
        velocity = 2 * diff(self, t) * self.conjugate()
        return Vector(
            velocity.x,
            velocity.y,
            velocity.z
        )
    
    def local_angular_velocity(self, t):
        velocity = 2 * self.conjugate() * diff(self, t)
        return Vector(
            velocity.x,
            velocity.y,
            velocity.z
        )
    
    def _repr_latex_(self):
        return r"$\left[{}, {}, {}, {}\right]$".format(
            printing.latex(self.w),
            printing.latex(self.x),
            printing.latex(self.y),
            printing.latex(self.z),
        )
    
    def _latex(self, printer):
        return r"\left[{}, {}, {}, {}\right]".format(
            printing.latex(self.w),
            printing.latex(self.x),
            printing.latex(self.y),
            printing.latex(self.z),
        )
    
    def simplify(self):
        return Quaternion(
            simplify(self.w),
            simplify(self.x),
            simplify(self.y),
            simplify(self.z)
        )


class Transform(Expr):
    def __init__(self, translation: Vector, rotation: Quaternion):
        super(Transform, self).__init__()
        self.translation = translation
        self.rotation = rotation

    @staticmethod
    def identity():
        return Transform(
            Vector.zero(),
            Quaternion.identity()
        )

    def inverse(self):
        conj = self.rotation.conjugate()
        return Transform(
            conj * (-1 * self.translation),
            conj
        )

    @staticmethod
    def lerp(start, end, t):
        return Transform(
            Vector.lerp(start.translation, end.translation, t),
            Quaternion.slerp(start.rotation, end.rotation, t)
        )

    def __str__(self):
        return "[\n\t{}\n\t{}\n]".format(str(self.translation), str(self.rotation))

    def __repr__(self):
        return "Transform[\n\t{}\n\t{}\n]".format(str(self.translation), str(self.rotation))

    def __add__(self, other):
        if isinstance(other, Transform):
            return Transform(
                self.translation + self.rotation * other.translation,
                self.rotation * other.rotation
            )
        if isinstance(other, Vector):
            return self.translation + self.rotation * other
        
    def _repr_latex_(self):
        return r"$\left[{}, {}\right]$".format(
            printing.latex(self.translation),
            printing.latex(self.rotation),
        )
    
    def simplify(self):
        return Transform(
            simplify(self.translation),
            simplify(self.rotation)
        )


Зададим основные переменные:
- время
- ускорение свободного падения
- моменты инерции звеньев и объекта
- центры масс звеньев и объекта
- массы звеньев и объекта
- длины звеньев
- обобщенные координаты

In [ ]:
t = Symbol("t")
g = Symbol("g")
j = symbols("J_0:4", cls=MatrixSymbol, n=3, m=3)
cmx = symbols("c_mx0:4")
cmy = symbols("c_my0:4")
cmz = symbols("c_mz0:4")
m = symbols("m_0:4")
l = symbols("l_0:3")
q_ = symbols("q_0:3", cls=Function)
q = [q_[0](t), q_[1](t), q_[2](t)]

Укажем положения звеньев:

In [ ]:
def transform0(q):
    return Transform(
        Vector(0, 0, 0),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )

def transform1(q):
    return transform0(q) + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[1], Vector(1, 0, 0))
    )

def transform2(q):
    return transform1(q) + Transform(
        Vector(0, 0, l[1]),
        Quaternion.from_angle_axis(q[2], Vector(1, 0, 0))
    )

def transform3(q):
    return transform2(q) + Transform(
        Vector(0, 0, l[2]),
        Quaternion.identity()
    )

И положения центров масс:

In [ ]:
def mass_position0(q):
    return transform0(q) + Vector(cmx[0], cmy[0], cmz[0])

def mass_position1(q):
    return transform1(q) + Vector(cmx[1], cmy[1], cmz[1])

def mass_position2(q):
    return transform2(q) + Vector(cmx[2], cmy[2], cmz[2])

def mass_position3(q):
    return transform3(q) + Vector(cmx[3], cmy[3], cmz[3])

Рассчитаем кинетическую энергию:

In [ ]:
def kinetic_energy0(q):
    return mass_position0(q).velocity(t) ** 2 * m[0] / 2 +\
        transform0(q).rotation.local_angular_velocity(t) ** 2 * j[0] / 2

def kinetic_energy1(q):
    return mass_position1(q).velocity(t) ** 2 * m[1] / 2 +\
        transform1(q).rotation.local_angular_velocity(t) ** 2 * j[1] / 2

def kinetic_energy2(q):
    return mass_position2(q).velocity(t) ** 2 * m[2] / 2 +\
        transform2(q).rotation.local_angular_velocity(t) ** 2 * j[2] / 2

def kinetic_energy3(q):
    return mass_position3(q).velocity(t) ** 2 * m[3] / 2 +\
        transform3(q).rotation.local_angular_velocity(t) ** 2 * j[3] / 2

def total_kinetic_energy(q):
    return kinetic_energy0(q) +\
        kinetic_energy1(q) +\
        kinetic_energy2(q) +\
        kinetic_energy3(q)

И потенциальную:

In [ ]:
def potential_energy0(q):
    return mass_position0(q).z * m[0]

def potential_energy1(q):
    return mass_position1(q).z * m[1]

def potential_energy2(q):
    return mass_position2(q).z * m[2]

def potential_energy3(q):
    return mass_position3(q).z * m[3]

def total_potential_energy(q):
    return potential_energy0(q) +\
        potential_energy1(q) +\
        potential_energy2(q) +\
        potential_energy3(q)

Дальше все просто:

In [ ]:
def lagrangian(q):
    return total_kinetic_energy(q) - total_potential_energy(q)

In [ ]:
def force_calculation0(q):
    lgr = lagrangian(q)
    return lgr.diff(q[0].diff(t)).diff(t) - lgr.diff(q[0])

def force_calculation1(q):
    lgr = lagrangian(q)
    return lgr.diff(q[1].diff(t)).diff(t) - lgr.diff(q[1])

def force_calculation2(q):
    lgr = lagrangian(q)
    return lgr.diff(q[2].diff(t)).diff(t) - lgr.diff(q[2])

In [ ]:
force0 = force_calculation0(q)
force1 = force_calculation1(q)
force2 = force_calculation2(q)

Посмотрим силы в простом случае:
- $q_0(t) = t$
- $q_1(t) = \frac{\pi}{2}$
- $q_1(t) = -\frac{\pi}{2}$

In [ ]:
force0.replace(q[0], t).replace(q[1], pi/2).replace(q[2], -pi/2).simplify()

In [ ]:
force1.replace(q[0], t).replace(q[1], pi/2).replace(q[2], -pi/2).simplify()

In [ ]:
force2.replace(q[0], t).replace(q[1], pi/2).replace(q[2], -pi/2).simplify()